In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner import HyperParameters

import autokeras as ak

from cerebro.nas.hphpmodel import HyperHyperModel

In [2]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node = ak.ClassificationHead(output_node1)

am = HyperHyperModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

In [3]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

from cerebro.backend import SparkBackend
from cerebro.storage import LocalStore

backend = SparkBackend(spark_context=sc, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

am.resource_bind(backend=backend, store=store)

21/11/13 21:00:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/13 21:00:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


CEREBRO => Time: 2021-11-13 21:00:52, Running 1 Workers


In [4]:
am.tuner_bind("gridsearch", hyperparameters=None)

INFO:tensorflow:Reloading Oracle from existing project ./test/oracle.json


In [ ]:
oracle = am.tuner.oracle

hps = HyperParameters()
hps.Choice("lr", values=[1,2])
hps.Choice("bs", values=[3,4])
hps.Choice("wtv", values=["a", "b", "c"])
hps.space

In [8]:
oracle._grid_generation(hps)

[{'lr': 1, 'bs': 3},
 {'lr': 1, 'bs': 4},
 {'lr': 2, 'bs': 3},
 {'lr': 2, 'bs': 4}]

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = np.array(tf.one_hot(y_train, len(np.unique(y_train))))

am.test_tuner_space(x=x_train, y=y_train)

am.tuner.search_space_summary()

2021-11-13 21:00:52.856851: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Search space summary
Default search space size: 23
learning_rate (Fixed)
{'conditions': [], 'value': 0.0001}
conv_block_1/kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
conv_block_1/separable (Boolean)
{'default': False, 'conditions': []}
conv_block_1/max_pooling (Boolean)
{'default': True, 'conditions': []}
conv_block_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
conv_block_1/num_blocks (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
conv_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2], 'ordered': True}
conv_block_1/filters_0_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_0_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
conv_block_1/filters_1_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 3

In [9]:
am.tuner.oracle.init_search_space()

KeyboardInterrupt: 

In [ ]:
am.tuner.oracle.search_space

In [ ]:
am.tuner